In [118]:
import pandas as pd


In [170]:
def read_conll_data(path, tokenizer=None):
    with open(path,'r') as input_text:
        text = input_text.read().strip()
    
    docs = text.split('\n\n')
    data = []
    labels = set()
    
    for i,d in enumerate(docs):

        record = dict()
        record['id'] = i
        elements = [e.split(' ') for e in d.split('\n')]
        record['words'] = [e[0] for e in elements]
        record['ner_tags'] = [e[-1] for e in elements]
       
        
        
        labels.update(record['ner_tags'])
        data.append(record)
    labels2id = dict()
    labels2id = {l:i+1 for i,l in enumerate(labels) if l != 'O'}
    labels2id['O'] = 0
    
    print(labels2id)
    for record in data:
        record['ner_tags'] = [labels2id[l] for l in record['ner_tags']]
    # for record in data:
        
    #     tokens,token_labels = [],[]
    #     for word,label in zip(record['words'],record['ner_tags']):
    #         word_tokens = tokenizer.tokenize(word)

    #         tokens.extend(word_tokens)
    #         token_labels.extend([labels2id[label]]*len(word_tokens))
    #     record['tokens'] = tokens
    #     record['ner_tags'] = token_labels
             
    return data,labels2id

    




In [171]:
from transformers import AutoTokenizer
checkpoint = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
data, label2id = read_conll_data('data/project-5-at-2023-06-13-13-05-b9955df3.conll', tokenizer=tokenizer)

loading configuration file config.json from cache at /Users/kasparbeelen/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at /Users/kasparbeelen/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file tokenizer.json from cache at /Users/kasparbeelen/.cache/huggingface/

{'I-Object': 1, 'B-Object': 2, 'O': 0}


In [172]:
label2id

{'I-Object': 1, 'B-Object': 2, 'O': 0}

In [173]:
from datasets import load_dataset
import datasets
data = datasets.Dataset.from_pandas(pd.DataFrame(data=data))

In [174]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["words"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"ner_tags"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:  # Set the special tokens to -100.
            if word_idx is None:
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [175]:
tokenized_data = data.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [176]:
tokenized_data

Dataset({
    features: ['id', 'words', 'ner_tags', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 970
})

In [177]:
tokenized_data = tokenized_data.train_test_split(test_size=.2)

In [180]:
tokenized_data['test'][0]

{'id': 646,
 'words': ['LATER',
  'FROM',
  'AMERICA',
  '.',
  '-',
  '..',
  'llinv',
  'YORK',
  ',',
  'Oct.',
  '7',
  '(',
  'Evening',
  '.',
  ')',
  'Mr.',
  'Stanton',
  'reports',
  'tha',
  '-',
  't',
  'the',
  'operations',
  'are',
  'progressing',
  'satisfactorily',
  'before',
  'Riehn.s.ond',
  'and',
  'Petersburg',
  '.',
  'No',
  'engagement',
  'has',
  'taken',
  "p'ace",
  "'",
  'shy',
  '.',
  ',',
  'Saturday',
  '.',
  'Both',
  'belligerents',
  'were',
  'fortifying',
  '.',
  'It',
  'is',
  "i'ported",
  'that',
  'Lee',
  'is',
  'concentrating',
  'on',
  'Gr',
  'trit',
  "'s",
  'left',
  '.',
  'Thn',
  'federal',
  'loss',
  'north',
  'and',
  'south',
  'of',
  'the',
  'James',
  'River',
  'on',
  'Fridly',
  'and',
  'Saturday',
  'Is',
  'estimated',
  'at',
  '4,000',
  '.',
  'The',
  'North‘rn',
  'papers',
  'contain',
  'a',
  'report',
  'that',
  'the',
  'larger',
  'portion',
  'of',
  'Grant',
  "'s",
  'corps',
  'has',
  're-',

In [178]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [181]:
import evaluate

seqeval = evaluate.load("seqeval")

In [182]:
label_list = list(label2id.keys())
label_list,label2id

(['I-Object', 'B-Object', 'O'], {'I-Object': 1, 'B-Object': 2, 'O': 0})

In [183]:
import numpy as np

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)

    true_predictions = [
        [label_list[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [label_list[l] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]

    results = seqeval.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [184]:
id2label = {i:l for l,i in label2id.items()}

In [185]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer

model = AutoModelForTokenClassification.from_pretrained(
    checkpoint, num_labels=len(label2id), id2label=id2label, label2id=label2id
)

loading configuration file config.json from cache at /Users/kasparbeelen/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "I-Object",
    "2": "B-Object"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-Object": 2,
    "I-Object": 1,
    "O": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /Users/kasparbeelen/.cache/huggingface/hub/models--dist

In [186]:
training_args = TrainingArguments(
    output_dir="object-detector",
    learning_rate=3e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=5,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    #push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_data["train"],
    eval_dataset=tokenized_data["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()
trainer.save_model()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
The following columns in the training set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, words, id. If ner_tags, words, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
/Users/kasparbeelen/anaconda3/envs/ce/lib/python3.9/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 776
  Num Epochs = 5
  Instanta

  0%|          | 0/245 [00:00<?, ?it/s]

You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
The following columns in the evaluation set don't have a corresponding argument in `DistilBertForTokenClassification.forward` and have been ignored: ner_tags, words, id. If ner_tags, words, id are not expected by `DistilBertForTokenClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 194
  Batch size = 16


  0%|          | 0/13 [00:00<?, ?it/s]

Saving model checkpoint to object-detector/checkpoint-49
Configuration saved in object-detector/checkpoint-49/config.json


...

Saving model checkpoint to object-detector/checkpoint-245
Configuration saved in object-detector/checkpoint-245/config.json


{'eval_loss': 0.005650777369737625, 'eval_precision': 0.8777506112469438, 'eval_recall': 0.8527315914489311, 'eval_f1': 0.8650602409638554, 'eval_accuracy': 0.9988484790327223, 'eval_runtime': 22.981, 'eval_samples_per_second': 8.442, 'eval_steps_per_second': 0.566, 'epoch': 5.0}


Model weights saved in object-detector/checkpoint-245/pytorch_model.bin
tokenizer config file saved in object-detector/checkpoint-245/tokenizer_config.json
Special tokens file saved in object-detector/checkpoint-245/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from object-detector/checkpoint-245 (score: 0.005650777369737625).
Saving model checkpoint to object-detector
Configuration saved in object-detector/config.json


{'train_runtime': 1956.328, 'train_samples_per_second': 1.983, 'train_steps_per_second': 0.125, 'train_loss': 0.027506915890440648, 'epoch': 5.0}


Model weights saved in object-detector/pytorch_model.bin
tokenizer config file saved in object-detector/tokenizer_config.json
Special tokens file saved in object-detector/special_tokens_map.json


In [189]:
%%bash
ls object-detector

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
checkpoint-147
checkpoint-196
checkpoint-245
checkpoint-49
checkpoint-98
config.json
pytorch_model.bin
special_tokens_map.json
tokenizer.json
tokenizer_config.json
training_args.bin
vocab.txt


In [196]:
from transformers import pipeline

classifier = pipeline("ner", model="object-detector")

loading configuration file object-detector/config.json
Model config DistilBertConfig {
  "_name_or_path": "object-detector",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "I-Object",
    "2": "B-Object"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-Object": 2,
    "I-Object": 1,
    "O": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading configuration file object-detector/config.json
Model config DistilBertConfig {
  "_name_or_path": "object-detector",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],

In [198]:
text = ' '.join(tokenized_data['test'][2]['words'])
#text = 'I will take the train to London.'

results = classifier(text)
results

[{'entity': 'B-Object',
  'score': 0.9890133,
  'index': 65,
  'word': 'yard',
  'start': 297,
  'end': 301},
 {'entity': 'B-Object',
  'score': 0.99073935,
  'index': 124,
  'word': 'body',
  'start': 596,
  'end': 600}]

loading configuration file object-detector/config.json
Model config DistilBertConfig {
  "_name_or_path": "object-detector",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "O",
    "1": "I-Object",
    "2": "B-Object"
  },
  "initializer_range": 0.02,
  "label2id": {
    "B-Object": 2,
    "I-Object": 1,
    "O": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading configuration file object-detector/config.json
Model config DistilBertConfig {
  "_name_or_path": "object-detector",
  "activation": "gelu",
  "architectures": [
    "DistilBertForTokenClassification"
  ],

In [ ]:
from collections import defaultdict
def model_to_labelstudio(text, classifier):
    results = classifier(text)

    record = defaultdict(dict)
    record['data']['text'] = text
    record['data']['num_objects'] = len(results)

    predictions = []
    
    

In [ ]:
def predictions_

In [8]:
text = open('data/project-5-at-2023-06-13-10-36-5850e0ef.conll').read().strip()
docs = text.split('\n\n')

In [9]:
len(docs)

4

In [11]:
tokens = docs[0].split('\n')

['THE', '-X-', '_', 'O']